In [1]:
import pandas as pd
import numpy as np
from sklearn import svm

#!pip install bayesian-optimization
#!pip install torch
#!pip install pyro


|Variable|Erklärung|Anmerkung|
|:---|:---|:---|
Age|Age of the patient|
Sex|Sex of the patient
exang|exercise induced angina (1 = yes; 0 = no)|Hat man Engegefühl wenn man Sport macht?
caa|number of major vessels (0-3)|weglassen, da Beschreibung uneindeutig, Skala passt auch nicht [0-3] 
cp|Chest Pain type chest pain type|Value 1: typical angina<br>Value 2: atypical angina<br>Value 3: non-anginal pain<br>Value 4: asymptomatic
trtbps|resting blood pressure (in mm Hg): Blutdruck bei Ruhe
chol|cholestoral in mg/dl fetched via BMI sensor|Cholesterinspiegel
fbs|(fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)|Erhöhter Blutzucker
rest_ecg|resting electrocardiographic results|Value 0: normal<br>Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)<br>Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
thalach|maximum heart rate achieved: Puls
target|0= less chance of heart attack 1= more chance of heart attack

In [2]:
# Datenset laden
# https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset
fulldata = pd.read_csv("heart.csv")

In [3]:
# Datentypen anpassen
cols = [
    "sex",
    "cp",
    "fbs",
    "restecg"
]

for col in cols:
    fulldata[col] = fulldata[col].astype("category")
    
cols = [
    "exng",
    "output"
]

for col in cols:
    fulldata[col] = fulldata[col].astype("bool")

In [4]:
#https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset
fulldata = pd.read_csv("heart.csv")
#data = fulldata.drop(['cp', 'restecg', 'output'], axis=1)
data = fulldata.drop(['output'], axis=1)
output = fulldata['output']
data

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


In [3]:
from sklearn.model_selection import train_test_split

# load data
X, y = data, output


# split the data into 2 sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(227, 13) (227,)
(76, 13) (76,)


In [4]:
from sklearn.model_selection import cross_val_score

result_array = []
# warum keine precomputed method?
methods = ["linear", "poly", "rbf", "sigmoid"]
i = 0
method = methods[i] 
k = 10
for element in methods:
    print(method)
    clf = svm.SVC(kernel=method, C=1, random_state=42)
    scores = cross_val_score(clf, data, output, cv=k)
    result_array.append((scores.max(), method))
    if i < 3:
        i = i + 1
    method = methods[i] 
result_array

linear
poly
rbf
sigmoid


[(0.9333333333333333, 'linear'),
 (0.8666666666666667, 'poly'),
 (0.8, 'rbf'),
 (0.5666666666666667, 'sigmoid')]

k = 10
clf = svm.SVC(kernel="linear", C=1, random_state=42)
scores = cross_val_score(clf, data, output, cv=k)
scores

In [6]:
#clf = svm.SVC(kernel=linear, C=1, random_state=42)
#scores = cross_val_score(clf, data, output, cv=k)

param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']}]

from sklearn.model_selection import GridSearchCV

# create a SVC and optimize it with a grid search
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(data, output)

# get the best estimator
clf.best_estimator_

SVC(C=1, kernel='linear')

In [7]:
k = 10
clf = svm.SVC(kernel="linear", C=0.0001, random_state=42)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))


0.6973684210526315


In [20]:
def black(C):
    # C: SVC hyper parameter to optimize for.
    #model = SVC(C = C, kernel="linear" )
    model = svm.SVC(kernel="linear", C=C, random_state=42)
    model.fit(X_train, y_train)
    
    #from sklearn.metrics import accuracy_score 
    #data["predicted"] = model.predict(data) 
    #f = accuracy_score(y_true = fulldata["output"], y_pred = data["predicted"])
    
    
    
    #f = model.score(X_test, y_test)
    #model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    #y_score = model.decision_function(X_test_scaled)
    #y_score = model.decision_function(X_test)
    #f = roc_auc_score(y_test, y_score)
    #y_score = model.decision_function(X_test)
    f = model.score(X_train, y_train)
    return f

In [25]:
black(83.34510871314733)

0.8722466960352423

In [ ]:
data

In [ ]:
print(np.array(data))

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization, UtilityFunction
import warnings
warnings.filterwarnings("ignore")


scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(np.array(X_train))
X_test_scaled = scaler.transform(np.array(X_test))
# Define the black box function to optimize.

def black_box_function(C):
    # C: SVC hyper parameter to optimize for.
    #model = SVC(C = C, kernel="linear" )
    
    model = svm.SVC(kernel="linear", C=C, random_state=42)
    model.fit(X_train, y_train)
    
    #from sklearn.metrics import accuracy_score 
    #fulldata["predicted"] = model.predict(data) 
    #f = accuracy_score(y_true = fulldata["output"], y_pred = fulldata["predicted"])
    #model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    #y_score = model.decision_function(X_test_scaled)
    #y_score = model.decision_function(X_test)
    #f = roc_auc_score(y_test, y_score)
    #y_score = model.decision_function(X_test)
    f = model.score(X_train, y_train)
    return f

def black_box_functionorigi(C):
    # C: SVC hyper parameter to optimize for.
    model = SVC(C = C, kernel="linear", random_state=42 )
    
    #model.fit(X_train_scaled, y_train)
    model.fit(X_train, y_train)
    #y_score = model.decision_function(X_test_scaled)
    y_score = model.decision_function(X_test)
    f = roc_auc_score(y_test, y_score)
    #y_score = model.decision_function(X_test)
    #f = model.score(X_train, y_train)
    return f
# Set range of C to optimize for.
# bayes_opt requires this to be a dictionary.
pbounds = {"C": [1, 100]}
# Create a BayesianOptimization optimizer,
# and optimize the given black_box_function.
optimizer = BayesianOptimization(f = black_box_function,
                                 pbounds = pbounds, verbose = 2,
                                 random_state = 1)
optimizer.maximize(init_points = 10, n_iter = 10)
print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.8458   |  42.29    |
|  2        |  0.8502   |  72.31    |
|  3        |  0.8546   |  1.011    |
|  4        |  0.8458   |  30.93    |
|  5        |  0.8502   |  15.53    |
|  6        |  0.859    |  10.14    |
|  7        |  0.8546   |  19.44    |
|  8        |  0.8502   |  35.21    |
|  9        |  0.8414   |  40.28    |
|  10       |  0.8546   |  54.34    |
|  11       |  0.8546   |  7.913    |
|  12       |  0.8634   |  11.97    |
|  13       |  0.8458   |  23.22    |
|  14       |  0.859    |  13.0     |
|  15       |  0.837    |  58.71    |
|  16       |  0.8414   |  50.8     |
|  17       |  0.8414   |  4.252    |
|  18       |  0.8634   |  78.7     |
|  19       |  0.8634   |  80.6     |
|  20       |  0.8722   |  83.35    |
Best result: {'C': 83.34510871314733}; f(x) = 0.8722466960352423.


In [ ]:
from sklearn import svm
from matplotlib import pyplot as plt
# we just use 2 dimensions here because we want to visualize the result

model = svm.SVC(kernel='linear', C=1)
#svm.SVC(kernel='rbf', C=1, gamma=.7)
#models = (clf.fit(data["age"], data["output"]) for clf in models)
fitted = model.fit(data["age"], data["output"])


In [ ]:
# define helper functions
def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
# title for the plots
titles = ('SVC with linear kernel',
          'SVC with RBF kernel')

# Set-up 1x2 grid for plotting.
fig, sub = plt.subplots(1, 2, figsize=(15, 7))
plt.subplots_adjust(wspace=0.4, hspace=0.4)

X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)

for clf, title, ax in zip(models, titles, sub.flatten()):
    plot_contours(ax, clf, xx, yy,
                  cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Petal length')
    ax.set_ylabel('Sepal width')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)
